# Checkpoint (model copies)

- storing intermediate parameters (checkpoints), as they could be better parameters. can restart from the good checkpoint and find better parameters.

say the loss is lower in an intermediate epoch/step - **store copies of parameters** at those points.  

large number of parameters => high chance for oscillation of loss.

In [1]:
import numpy as np
import torch

import matplotlib.pyplot as plt

In [2]:
import torchvision
import torchvision.transforms as transforms

In [3]:
import torch.nn as nn
import torch.optim as optim

from torchvision import models

import os


In [5]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Dataset

In [6]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

# crop - portion of the image instead of shrinking them unequally along differet dim
# crop from a random location.
# here 32x32 to 224x224 - blow up.
# normalize for images also like other data
# mean, SD in 3 channels.
# 0.5,0.5 -> most values b/w 0,1
# ideally we should find the mean, SD(of train data) and then use those values
# it will be available in the dataset website


transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

# here we are using the same set of transforms for both test and train
# but in general we might want different ones.
# eg: data augmentation - (eg. flip horizontally) - in train
# but not in test.

# should use the same mean, SD for normalizing test data also.
# same as that of train data.

###############################################################

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=False, 
                                        transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                        download=False, 
                                        transform=transform_test)

### Utils

In [7]:
# function to find accuracy - given dataloader, model
def evaluation(dataloader, model):
    model.eval()
    # model be already in device
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        # move data to device (in batches)
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

# device is global

### Model (VGG from torchvision.models)

In [8]:
vgg = models.vgg16_bn()

### Checkpoint

In [9]:
import copy

# allows to make deep copies of structures(objects) in python

using copy.deepcopy  
state_dict => access to all parameters.  
> best_model = copy.deepcopy(vgg.state_dict())   

adding this to above train:

## Train (with checkpoint)

In [ ]:
# dataloader
batch_size = 16
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

# build model
vgg = vgg.to(device)

# loss function
loss_fn = nn.CrossEntropyLoss()

# build opt
opt = optim.SGD(vgg.parameters(), lr=0.05)

#########################################################################################

loss_per_epoch_arr = []
num_epochs = 1

min_loss = 1000

# number of batches in dataset
num_steps = np.ceil(50000/batch_size)

for epoch in range(num_epochs):

    for i, data in enumerate(trainloader, 0):
        
        # data
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad() # reset gradients
        
        # forward pass
        outputs = vgg(inputs)
        
        # loss
        loss = loss_fn(outputs, labels)
        
        # compute gradients
        loss.backward()
        
        # update parameters
        opt.step()
        
        # save one with min loss.
        if min_loss > loss.item():
            min_loss = loss.item()
            # using copy.deepcopy
            # state_dict - access to all parameters.
            best_model = copy.deepcopy(vgg.state_dict())
            print('Min loss %0.2f' % min_loss)
        
        # DELETING CURRENT BATCH TO SAVE MEMORY - AS ITS NOT NEEDED AGAIN
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
        # print loss every 100 steps
        if i % 100 == 0:
            print('Step: %d/%d, Loss: %0.2f' % (i, num_steps, loss.item()) )
        
    loss_per_epoch_arr.append(loss.item())
    
    # Evaluation after each epoch      
    # no grad is there in evaluation fucntion. 
    # can add here also to be safe.
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
        epoch, num_epochs, 
        evaluation(testloader, vgg), evaluation(trainloader, vgg)))
    
    
plt.plot(loss_per_epoch_arr)
plt.show()

In [ ]:
vgg.load_state_dict(best_model) # load the copied state_dict into the model
# overwrites the parameters with those.

# print(evaluation(trainloader, vgg), evaluation(testloader, vgg))
print(evaluation(testloader, vgg))

typically - we start from the best model and rerun the training. - learn further from that point (with a **reduced learning rate**, .. )